In [1]:
import tensorflow as tf

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = 'D:/belajar/Data/Flower/training'
train_dir

'D:/belajar/Data/Flower/training'

In [16]:
# data augmnetasi and normalization
train_datagen = ImageDataGenerator(
    rescale = 1./225,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.2
)
train_datagen

In [20]:
# create data generator (train)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    subset ='training',
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)
train_generator

Found 2198 images belonging to 5 classes.


In [21]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    subset = 'validation',
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

Found 548 images belonging to 5 classes.


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax') # output layar with softmax activation
])

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor ='loss', patience=3)


In [27]:
# train to model
from tensorflow.keras.callbacks import EarlyStopping
history = model.fit(
    train_generator, 
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // validation_generator.batch_size,
    callbacks = [early_stopping]
)

Epoch 1/10
68/68 [==============================] - 35s 493ms/step - loss: 1.4377 - accuracy: 0.3652 - val_loss: 1.2734 - val_accuracy: 0.4743
Epoch 2/10
68/68 [==============================] - 38s 555ms/step - loss: 1.1749 - accuracy: 0.5092 - val_loss: 1.0934 - val_accuracy: 0.5643
Epoch 3/10
68/68 [==============================] - 33s 489ms/step - loss: 1.0729 - accuracy: 0.5803 - val_loss: 1.0999 - val_accuracy: 0.5699
Epoch 4/10
68/68 [==============================] - 37s 552ms/step - loss: 1.0398 - accuracy: 0.5886 - val_loss: 1.0524 - val_accuracy: 0.5938
Epoch 5/10
68/68 [==============================] - 44s 643ms/step - loss: 0.9978 - accuracy: 0.6090 - val_loss: 0.9843 - val_accuracy: 0.6213
Epoch 6/10
68/68 [==============================] - 41s 609ms/step - loss: 0.9608 - accuracy: 0.6297 - val_loss: 0.9832 - val_accuracy: 0.6176
Epoch 7/10
68/68 [==============================] - 50s 727ms/step - loss: 0.9247 - accuracy: 0.6500 - val_loss: 0.9092 - val_accuracy: 0.6397

In [30]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss:.2f}')
print(f'Validation Accuracy: {val_accuracy * 100:.2f}')

18/18 [==============================] - 4s 200ms/step - loss: 0.8898 - accuracy: 0.6588
Validation Loss: 0.89
Validation Accuracy: 65.88


In [31]:
loss, accuracy = model.evaluate(train_generator)
print(f'Validation Loss: {loss:.2f}')
print(f'Validation Accuracy: {accuracy * 100:.2f}')

69/69 [==============================] - 15s 212ms/step - loss: 0.7669 - accuracy: 0.7079
Validation Loss: 0.77
Validation Accuracy: 70.79


In [33]:
# mencoba menambha input baru

from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocessing_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 225.0
    return img_array

In [41]:
# memprediksi kelas gambar baru

def predict_image_class(img_path, model, class_indices):
    img_array = load_and_preprocessing_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # mapping index ke kelas label
    class_labels = {v: k for k, v in class_indices.items()}
    return class_labels[predicted_class[0]]


In [43]:
img_path = "D:/belajar/Data/Flower/rose kuning.jpg"
predicted_class = predict_image_class(img_path, model, train_generator.class_indices)
print(f'Predicted class : {predicted_class}')

1/1 [==============================] - 0s 25ms/step
Predicted class : tulip
